In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Vollmondgeburten

## Aufgabe
Kürzlich hat mir jemand gesagt, dass nach dem Vollmond besonders viele Babies zur Welt kämen.
Der Mond scheint durchaus Einfluss auf uns zu haben, aber diese Aussage schien mir doch etwas
gewagt. Da zum Glück Daten zur täglichen Anzahl der Geburten schon länger erfasst werden,
können wir diese Aussage relativ einfach überprüfen. Das Bundesamt für Statistik stellt Daten
mit der täglichen Geburtenzahl ab dem Jahr 1969 zur Verfügung. Sie finden die Excel-Datei
’BFS_VitalStatistics.xlsx’ bei den mitgelieferten Dateien.

## a)

Lesen Sie die Datei in ein Data Frame und bringen Sie die Daten in das Format

Kanton, Jahr, Monat, Tag, Anzahl, Geburten

Stellen Sie sicher, dass Sie alle Verunreinigungen entfernt haben und dass keine fehlenden Werte vorkommen. Setzen Sie den Datentyp `float` für alle Spalten ausser `Kanton` (Pandas hat Probleme mit fehlenden Werten für den Datentyp `int`).

In [ ]:
columns_to_read = list(range(38, 573))  # AM to PQ
columns_to_read.append(1)  # Column A

df = pd.read_excel('BFS_VitalStatistics.xlsx', header=[3], usecols='A, D, AN:PQ')
df = df.rename(columns={'Unnamed: 0': 'Jahr', 'Unnamed: 3': 'Kanton'})
df['Jahr'] = df['Jahr'].fillna(method='ffill')
df = df.dropna(subset=['Kanton'])
df = df.filter(regex=r'^(?!.*indication).*$', axis=1)
df = df.filter(regex=r'^(?!.*total).*$', axis=1)

df

In [ ]:
df = df.melt(id_vars=['Jahr', 'Kanton'], var_name='TagMonat', value_name='Anzahl Geburten')

In [ ]:
df[['Tag', 'Monat']] = df['TagMonat'].str.split('.', expand=True)


In [ ]:
geburten = df[['Kanton','Jahr', 'Monat', 'Tag', 'Anzahl Geburten']]
geburten.loc[:, 'Anzahl Geburten'] = pd.to_numeric(df['Anzahl Geburten'], errors='coerce')
geburten

## b)

Leiten Sie nun aus den Spalten *Jahr*, *Monat* und *Tag* einen Datumstring ab und bringen Sie diesen in ein geeignetes Datumsformat. Beschränken Sie das Data Frame ausserdem auf Daten nur aus der ganzen Schweiz (Kanton='Switzerland'). Am Schluss sollte ihr Data Frame im folgenden Format vorliegen:

Datum, Anzahl Geburten

In [ ]:
geburten_ch = geburten[geburten['Kanton'] == 'Switzerland'].copy()
geburten_ch['Datum'] = pd.to_datetime(geburten_ch['Jahr'].astype(str) + '-' + geburten_ch['Monat'].astype(str) + '-' + geburten_ch['Tag'].astype(str), errors='coerce')
geburten_ch = geburten_ch.dropna(subset=['Datum'])
geburten_ch = geburten_ch.sort_values(by=['Datum'])
geburten_ch = geburten_ch.reset_index(drop=True)

geburten_ch = geburten_ch[['Datum', 'Anzahl Geburten']]
geburten_ch

## c)
Visualisieren Sie exemplarisch die Anzahl der Geburten pro Tag in der Schweiz (Kanton='Switzerland') im Jahr 2019. Stellen Sie dabei sicher, dass die $x$-Achse korrekt als Datum dargestellt wird.

In [ ]:
ch19 = geburten_ch[geburten_ch['Datum'].dt.year == 2019]
ch19


In [ ]:
plt.scatter(ch19['Datum'], ch19['Anzahl Geburten'])
plt.xlabel('Datum')
plt.ylabel('Anzahl Geburten')
plt.show()

plt.plot(ch19['Datum'], ch19['Anzahl Geburten'])
plt.xlabel('Datum')
plt.ylabel('Anzahl Geburten')
plt.show()

## d)

In der Datei `'mondphasen.csv'` finden Sie für jedes Datum seit 1970 die zugehörige Mondphase, benannt nach dem folgenden Schema:


Berechnen Sie nun die durchschnittliche Anzahl der Geburten in der Schweiz pro Mondphase seit 1970. Sehen Sie einen signifikanten Unterschied? (ohne auf statistische Tests zurückzugreifen) Natürlich müssten wir hier eine vertiefte Analyse durchführen, um sauber argumentieren zu können, aber einen ersten Eindruck können wir damit gut gewinnen.

In [ ]:
mp = pd.read_csv('mondphasen.csv')
mp['Datum'] = pd.to_datetime(mp['Datum'], errors='coerce')
geburten_ch['Datum'] = pd.to_datetime(geburten_ch['Datum'], errors='coerce')
mp

In [ ]:
gpm = pd.merge_ordered(geburten_ch, mp, on='Datum', how='left').dropna()

In [ ]:
gpm

## e)

Visualisieren Sie nun die durchschnittliche Anzahl der Geburten pro Wochentag über die Jahre hinweg. Finden Sie ein Muster?

In [ ]:
gpm['Wochentag'] = gpm['Datum'].dt.day_name()

In [ ]:
plt.scatter(gpm['Wochentag'], gpm['Anzahl Geburten'])
plt.xlabel('Wochentag')
plt.xticks(rotation=45)
plt.ylabel('Anzahl Geburten')
plt.show()

In [ ]:
mean_births_per_moonphase = gpm.groupby('Mondphase')['Anzahl Geburten'].mean()
print(mean_births_per_moonphase)
plt.scatter(gpm['Mondphase'], gpm['Anzahl Geburten'])
plt.xlabel('Mondphase')
plt.xticks(rotation=80)
plt.ylabel('Anzahl Geburten')
plt.show()